<a href="https://colab.research.google.com/github/Romema1/Romema1/blob/main/Copy_of_Crowdsourced_LMD_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Hybrid Optimization Framework for Crowdsourced Last-mile Delivery

This notebook contains the following workflow:

- ML model (XGBoost) to estimate acceptance probabilities \(P_{c,r}\).
- Geodesic distance matrices and optional **OSMnx** road network for rendering **real street paths**.
- Two ILP variants (PuLP):
  1. **Weighted objective**: maximize \(P_{c,r}\) minus distance penalty.
  2. **ε-constraint**: *minimize distance* subject to expected acceptances ≥ K.
- Heuristics & metaheuristics:
  - **Simulated Annealing (SA)** with multi-run harness.
  - **Adaptive Simulated Annealing (ASA)** with target acceptance and reheating.
  - **Genetic Algorithm (GA)** with multi-run harness.
  - **Adaptive GA (AGA)** with diversity/stagnation feedback.
- Folium map visualizations (straight lines **and** road-network polylines where available).
- Reproducible outputs saved to an `outputs/` folder (maps, CSV summaries, JSON best solutions).

> **Data files:**
> - `data/training_data.xlsx` — features with an `Output` column.
> - `data/Shgardi_data.xlsx` — sheets: `Customers`, `Couriers`, `Shgardi centers`.


**## Environment Setup and Library Imports**

In [2]:
# Install PuLP
%pip install pulp --quiet
%pip install osmnx networkx --quiet

# === Imports (single cell; no duplicates) ===
import os
import json
import time
import math
import copy
import random
from itertools import cycle
from pathlib import Path

import numpy as np
import pandas as pd

# ML
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Optimization
import pulp

# Distances / Mapping
from geopy.distance import geodesic
import folium

# Optional libs (graceful fallback if missing)
try:
    import osmnx as ox
    import networkx as nx
    OSMNX_AVAILABLE = True
except Exception:
    ox = None
    nx = None
    OSMNX_AVAILABLE = False

try:
    import joblib
    JOBLIB_AVAILABLE = True
except Exception:
    joblib = None
    JOBLIB_AVAILABLE = False

# Plotting (used only for SA boxplot; matplotlib is standard in most envs)
import matplotlib.pyplot as plt

# Make outputs directory
OUT_DIR = Path("outputs")
OUT_DIR.mkdir(exist_ok=True)

# === Configuration & file paths ===
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# Filenames (as per your originals)
ML_XLSX = "data/training_data.xlsx"
LOC_XLSX = "data/Shgardi_data.xlsx"

# Sheets
ML_SHEET = "Sheet1"
CUSTOMERS_SHEET = "Customers"
COURIERS_SHEET  = "Couriers"
CENTERS_SHEET   = "Shgardi centers"

# Balancing constraints
MIN_PER_CENTER = 12
MAX_PER_CENTER = 15

# Lambda for weighted objectives
LAMBDA_WEIGHT = 0.70

# City center for map/network build
CITY_CENTER = (26.3927, 50.1206)  # Dammam area
NETWORK_DIST_METERS = 100_000     # 100 km for "drive" (heavy); you can reduce

# Utility to consistently convert meters↔km
def m_to_km(m): return float(m)/1000.0
def km_to_m(km): return float(km)*1000.0

print("OSMnx available:", OSMNX_AVAILABLE)
print("joblib available:", JOBLIB_AVAILABLE)
print("Config loaded.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 3.0 MB/s eta 0:00:00
OSMnx available: True
joblib available: True
Config loaded.


In [3]:

# === Configuration & file paths ===
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# Filenames (as per your originals)
ML_XLSX = "data/training_data.xlsx"
LOC_XLSX = "data/Shgardi_data.xlsx"

# Sheets
ML_SHEET = "Sheet1"
CUSTOMERS_SHEET = "Customers"
COURIERS_SHEET  = "Couriers"
CENTERS_SHEET   = "Shgardi centers"

# Balancing constraints
MIN_PER_CENTER = 12
MAX_PER_CENTER = 15

# Lambda for weighted objectives
LAMBDA_WEIGHT = 0.70

# City center for map/network build
CITY_CENTER = (26.3927, 50.1206)  # Dammam area
NETWORK_DIST_METERS = 100_000     # 100 km for "drive" (heavy); you can reduce

# Utility to consistently convert meters↔km
def m_to_km(m): return float(m)/1000.0
def km_to_m(km): return float(km)*1000.0

print("Config loaded.")

Config loaded.


**## Configuration and File Paths**

In [4]:
# === Helpers: Data loading & extracts ===

def load_ml_data(path=ML_XLSX, sheet=ML_SHEET):
    df = pd.read_excel(path, sheet_name=sheet)
    if "Output" not in df.columns:
        raise ValueError("Expected 'Output' in ML sheet.")
    X = df.drop(columns=["Output"])
    y = df["Output"]
    return X, y, df

def load_location_data(path=LOC_XLSX):
    customers_df = pd.read_excel(path, sheet_name=CUSTOMERS_SHEET)
    couriers_df  = pd.read_excel(path, sheet_name=COURIERS_SHEET)
    centers_df   = pd.read_excel(path, sheet_name=CENTERS_SHEET)

    request_locations = list(zip(customers_df['Customer_Latitude'], customers_df['Customer_Longitude']))
    courier_locations = list(zip(couriers_df['Courier_Latitude'],  couriers_df['Courier_Longitude']))
    center_locations  = list(zip(centers_df['Shgardi_Latitude'],   centers_df['Shgardi_Longitude']))
    return customers_df, couriers_df, centers_df, request_locations, courier_locations, center_locations

def ensure_sizes(P_cr, num_couriers, num_requests):
    if P_cr.shape != (num_couriers, num_requests):
        raise ValueError(f"P_cr shape {P_cr.shape} does not match ({num_couriers}, {num_requests}).")

print("Helpers ready.")


Helpers ready.


## Part 1 — Train XGBoost and compute $P_{c,r}$

In [7]:
# Train/test split, scale, train XGBoost, compute acceptance probabilities
X, y, _ml_df = load_ml_data(path='/content/data for term project 242.xlsx')
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=RANDOM_SEED, stratify=y if len(np.unique(y))>1 else None
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_all_scaled   = scaler.transform(X)

xgb_model = XGBClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=30,
    subsample=1.0,
    colsample_bytree=1.0,
    eval_metric='logloss',
    random_state=RANDOM_SEED
)
xgb_model.fit(X_train_scaled, y_train)

if JOBLIB_AVAILABLE:
    joblib.dump({"model": xgb_model, "scaler": scaler}, OUT_DIR / "xgb_model_scaler.joblib")

# Load locations & sizes
customers_df, couriers_df, centers_df, request_locations, courier_locations, center_locations = load_location_data(path='/content/3 centers 5 drivers and 40 customers Shgardi_centers_customers_and_couriers_alkhobar copy.XLS')
num_couriers = len(courier_locations)   # expected 5
num_centers  = len(center_locations)    # expected 3
num_requests = len(request_locations)   # expected 40

# Build P_cr assuming the feature matrix rows are ordered by (courier, request)
accept_probs = xgb_model.predict_proba(X_all_scaled)[:, 1]
needed = num_couriers * num_requests
if len(accept_probs) < needed:
    raise ValueError(f"Not enough rows to build P_cr: need {needed}, have {len(accept_probs)}.")

P_cr = accept_probs[:needed].reshape((num_couriers, num_requests))
print("P_cr shape:", P_cr.shape)

P_cr shape: (5, 40)


## Part 2 — Distances and optional road network




In [8]:
# Geodesic distances (meters)
D_cc_prime = np.zeros((num_couriers, num_centers))  # courier -> center
for c in range(num_couriers):
    for cp in range(num_centers):
        D_cc_prime[c, cp] = geodesic(courier_locations[c], center_locations[cp]).meters

D_c_prime_r = np.zeros((num_centers, num_requests)) # center -> request
for cp in range(num_centers):
    for r in range(num_requests):
        D_c_prime_r[cp, r] = geodesic(center_locations[cp], request_locations[r]).meters

print("Distance matrices ready (meters).")

# Optional: build a drivable road network for path rendering
G = None
if OSMNX_AVAILABLE:
    try:
        G_dir = ox.graph_from_point(CITY_CENTER, dist=NETWORK_DIST_METERS, network_type="drive")
        G = G_dir.to_undirected()
        print("Road network built with OSMnx. Nodes:", len(G.nodes), "Edges:", len(G.edges))
    except Exception as e:
        print("[WARN] Could not build road network:", e)
else:
    print("OSMnx not available; road paths will fall back to straight-line polylines.")

Distance matrices ready (meters).


/usr/local/lib/python3.12/dist-packages/osmnx/_overpass.py:271: UserWarning: This area is 16 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


Road network built with OSMnx. Nodes: 144914 Edges: 216757


## Part 3 — ILP: Weighted objective (maximize acceptance − λ·distance)

In [ ]:
# ILP weighted objective
model_weighted = pulp.LpProblem("Driver_Center_Request_Assignment_Weighted", sense=pulp.LpMaximize)

# Decision variables
x_vars = {(c, cp): pulp.LpVariable(f"x_{c}_{cp}", cat=pulp.LpBinary)
          for c in range(num_couriers) for cp in range(num_centers)}
z_vars = {(c, cp, r): pulp.LpVariable(f"z_{c}_{cp}_{r}", cat=pulp.LpBinary)
          for c in range(num_couriers) for cp in range(num_centers) for r in range(num_requests)}

# Objective
model_weighted += pulp.lpSum(
    (P_cr[c, r] - LAMBDA_WEIGHT * (D_cc_prime[c, cp] + D_c_prime_r[cp, r])) * z_vars[(c, cp, r)]
    for c in range(num_couriers) for cp in range(num_centers) for r in range(num_requests)
)

# Constraints
# 1) Each center exactly one driver
for cp in range(num_centers):
    model_weighted += pulp.lpSum(x_vars[(c, cp)] for c in range(num_couriers)) == 1

# 2) Each driver at most one center
for c in range(num_couriers):
    model_weighted += pulp.lpSum(x_vars[(c, cp)] for cp in range(num_centers)) <= 1

# 3) Each request assigned exactly once
for r in range(num_requests):
    model_weighted += pulp.lpSum(z_vars[(c, cp, r)] for c in range(num_couriers) for cp in range(num_centers)) == 1

# 4) Linking z ≤ x
for c in range(num_couriers):
    for cp in range(num_centers):
        for r in range(num_requests):
            model_weighted += z_vars[(c, cp, r)] <= x_vars[(c, cp)]

# 5) Load balancing per center
for cp in range(num_centers):
    model_weighted += pulp.lpSum(z_vars[(c, cp, r)] for c in range(num_couriers) for r in range(num_requests)) >= MIN_PER_CENTER
    model_weighted += pulp.lpSum(z_vars[(c, cp, r)] for c in range(num_couriers) for r in range(num_requests)) <= MAX_PER_CENTER

# Optional driver capacity/time/distance constraints (kept relaxed; you can enable by setting arrays below)
ENABLE_DRIVER_LIMITS = False
T_c = [480]*num_couriers      # minutes
MaxRequests_c = [15]*num_couriers
Dmax_c_km = [80]*num_couriers

if ENABLE_DRIVER_LIMITS:
    # 6) Time availability: sum(Q_r * z) <= T_c per driver (assume each request is 10 minutes by default)
    Q_r = np.full(num_requests, 10)  # minutes per request
    for c in range(num_couriers):
        model_weighted += pulp.lpSum(Q_r[r] * z_vars[(c, cp, r)] for cp in range(num_centers) for r in range(num_requests)) <= T_c[c]

    # 7) Capacity: number of requests per driver
    for c in range(num_couriers):
        model_weighted += pulp.lpSum(z_vars[(c, cp, r)] for cp in range(num_centers) for r in range(num_requests)) <= MaxRequests_c[c]

    # 8) Distance limit: (meters) <= limit
    Dmax_c_m = [km_to_m(km) for km in Dmax_c_km]
    for c in range(num_couriers):
        model_weighted += pulp.lpSum((D_cc_prime[c, cp] + D_c_prime_r[cp, r]) * z_vars[(c, cp, r)]
                                     for cp in range(num_centers) for r in range(num_requests)) <= Dmax_c_m[c]

# Solve
status_w = model_weighted.solve(pulp.PULP_CBC_CMD(msg=False))
print("Weighted ILP status:", pulp.LpStatus[status_w])

# Extract solution
final_assignments_weighted = []
x_star = {(c, cp): pulp.value(var) for (c, cp), var in x_vars.items()}
for c in range(num_couriers):
    for cp in range(num_centers):
        if x_star[(c, cp)] and x_star[(c, cp)] > 0.5:
            for r in range(num_requests):
                if pulp.value(z_vars[(c, cp, r)]) > 0.5:
                    final_assignments_weighted.append((c, cp, r))

print("Assignments (c,cp,r):", len(final_assignments_weighted))

Weighted ILP status: Optimal
Assignments (c,cp,r): 40


**Part 4 — Distance-Based ILP (Exact solution) Optimization**

In [ ]:
import pulp
from geopy.distance import geodesic

# Infer sizes
num_couriers = len(courier_locations)
num_centers  = len(center_locations)
num_requests = len(request_locations)

# Build ILP: minimize (λ·distance − acceptance)
model = pulp.LpProblem("Vehicle_Routing_Assign", pulp.LpMinimize)

# Decision variables
x = {(c, cp): pulp.LpVariable(f"x_{c}_{cp}", cat="Binary")
     for c in range(num_couriers) for cp in range(num_centers)}
z = {(c, cp, r): pulp.LpVariable(f"z_{c}_{cp}_{r}", cat="Binary")
     for c in range(num_couriers) for cp in range(num_centers) for r in range(num_requests)}

lambda_w = 0.7  # distance weight

# Objective
model += pulp.lpSum(
    (
        lambda_w * (
            geodesic(courier_locations[c], center_locations[cp]).meters +
            geodesic(center_locations[cp], request_locations[r]).meters
        )
        - P_cr[c, r]
    ) * z[(c, cp, r)]
    for c in range(num_couriers)
    for cp in range(num_centers)
    for r in range(num_requests)
)

# Constraints
# 1) each center exactly one courier
for cp in range(num_centers):
    model += pulp.lpSum(x[(c, cp)] for c in range(num_couriers)) == 1

# 2) each courier at most one center
for c in range(num_couriers):
    model += pulp.lpSum(x[(c, cp)] for cp in range(num_centers)) <= 1

# 3) each request assigned exactly once
for r in range(num_requests):
    model += pulp.lpSum(z[(c, cp, r)]
                        for c in range(num_couriers)
                        for cp in range(num_centers)) == 1

# 4) linking: z ≤ x
for c in range(num_couriers):
    for cp in range(num_centers):
        for r in range(num_requests):
            model += z[(c, cp, r)] <= x[(c, cp)]

# 5) load balancing per center (12..15)
for cp in range(num_centers):
    model += pulp.lpSum(z[(c, cp, r)]
                        for c in range(num_couriers)
                        for r in range(num_requests)) >= 12
    model += pulp.lpSum(z[(c, cp, r)]
                        for c in range(num_couriers)
                        for r in range(num_requests)) <= 15

# Solve
model.solve(pulp.PULP_CBC_CMD(msg=False))
print("Optimal objective (minimized):", pulp.value(model.objective))

# Collect assignments
final_assignments = []
for c in range(num_couriers):
    for cp in range(num_centers):
        if pulp.value(x[(c, cp)]) > 0.5:
            for r in range(num_requests):
                if pulp.value(z[(c, cp, r)]) > 0.5:
                    final_assignments.append((c, cp, r))

print("Assigned triplets (c, cp, r):", len(final_assignments))


Optimal objective (minimized): 254431.39624023438
Assigned triplets (c, cp, r): 40


## Part 5 — Map visualizations (straight and road-network paths)

In [ ]:
import numpy as np
import folium
from geopy.distance import geodesic
from itertools import cycle
import osmnx as ox
import networkx as nx

def viz_assignments(assignments, title="Assignments Map",
                    use_road_network=True, zoom_start=12):
    """Return a folium.Map showing assignments as routes."""
    if not assignments:
        print("[viz] No assignments to visualize.")
        return None

    # Map center
    map_lat = np.mean([p[0] for p in courier_locations + center_locations + request_locations])
    map_lon = np.mean([p[1] for p in courier_locations + center_locations + request_locations])
    fmap = folium.Map(location=[map_lat, map_lon], zoom_start=zoom_start)
    fmap.get_root().html.add_child(folium.Element(f"<h4 style='text-align:center'>{title}</h4>"))

    # Markers
    assigned_drivers = {c for c, _, _ in assignments}
    for i, loc in enumerate(courier_locations):
        col = 'blue' if i in assigned_drivers else 'gray'
        folium.Marker(loc, popup=f"Driver C{i+1}", icon=folium.Icon(color=col, icon='user')).add_to(fmap)
    for j, loc in enumerate(center_locations):
        folium.Marker(loc, popup=f"Center {j+1}", icon=folium.Icon(color='green', icon='home')).add_to(fmap)
    for k, loc in enumerate(request_locations):
        folium.Marker(loc, popup=f"Request {k+1}", icon=folium.Icon(color='red', icon='info-sign')).add_to(fmap)

    # Colors per (courier, center)
    pair_to_reqs = {}
    for c, cp, r in assignments:
        pair_to_reqs.setdefault((c, cp), []).append(r)
    colors = cycle(['black','blue','green','purple','orange','cadetblue','darkred','darkpurple'])

    def nearest_node(pt):
        return ox.distance.nearest_nodes(G, pt[1], pt[0])

    for (c, cp), reqs in pair_to_reqs.items():
        clr = next(colors)

        # Simple visiting order: start at center, nearest-neighbor through its requests
        curr = center_locations[cp]
        ordered_pts = [courier_locations[c], center_locations[cp]]
        remaining = [request_locations[r] for r in reqs]
        while remaining:
            dists = [geodesic(curr, p).meters for p in remaining]
            idx = int(np.argmin(dists))
            curr = remaining.pop(idx)
            ordered_pts.append(curr)

        # Draw segments; use road network if available
        for u_pt, v_pt in zip(ordered_pts[:-1], ordered_pts[1:]):
            coords = [u_pt, v_pt]
            if use_road_network and 'G' in globals() and G is not None:
                try:
                    u = nearest_node(u_pt)
                    v = nearest_node(v_pt)
                    path = nx.shortest_path(G, u, v, weight='length')
                    coords = [(G.nodes[n]['y'], G.nodes[n]['x']) for n in path]
                except Exception:
                    coords = [u_pt, v_pt]
            folium.PolyLine(coords, color=clr, weight=4, opacity=0.8).add_to(fmap)

    return fmap

# Build and show the map
fmap = viz_assignments(final_assignments, title="Distance-Based ILP Assignments",
                       use_road_network=True, zoom_start=12)
fmap  # Jupyter/Colab will render this

# Optionally save to HTML:
# fmap.save("distance_ilp_map.html")


## Part 6 — Simulated Annealing (SA)

In [26]:
import random
import math
import copy

# Ensure these global variables are accessible within this cell's scope
num_couriers = len(courier_locations)
num_centers = len(center_locations)
num_requests = len(request_locations)

# SA fitness: same as weighted objective
def sa_fitness(center_assignment, request_assignment):
    global num_centers, P_cr, LAMBDA_WEIGHT, D_cc_prime, D_c_prime_r, MIN_PER_CENTER, MAX_PER_CENTER
    total = 0.0
    counts = [0]*num_centers
    for r, j in enumerate(request_assignment):
        if j < 0 or j >= num_centers:
            return -1e12  # invalid
        d = center_assignment[j]
        total += P_cr[d, r] - LAMBDA_WEIGHT * (D_cc_prime[d, j] + D_c_prime_r[j, r])
        counts[j] += 1
    # Penalties for balancing
    for j in range(num_centers):
        if counts[j] < MIN_PER_CENTER:
            total -= 1000 * (MIN_PER_CENTER - counts[j])
        elif counts[j] > MAX_PER_CENTER:
            total -= 1000 * (counts[j] - MAX_PER_CENTER)
    return total

def sa_random_candidate():
    global num_couriers, num_centers, num_requests
    centers = random.sample(range(num_couriers), num_centers)
    reqs = [random.randrange(num_centers) for _ in range(num_requests)]
    return centers, reqs

def sa_neighbor(centers, reqs):
    global num_couriers, num_centers, num_requests
    centers = centers.copy()
    reqs = reqs.copy()
    if random.random() < 0.3:
        # change one center's driver to an available one
        available = [d for d in range(num_couriers) if d not in centers]
        if available:
            idx = random.randrange(num_centers)
            centers[idx] = random.choice(available)
    else:
        i = random.randrange(num_requests)
        reqs[i] = random.randrange(num_centers)
    return centers, reqs

def simulated_annealing(initial, T_init=100, cooling=0.995, iterations=5000):
    global num_couriers, num_centers, num_requests
    centers, reqs = initial
    best_c, best_r = centers, reqs
    f_cur = sa_fitness(centers, reqs)
    f_best = f_cur
    T = T_init
    for _ in range(iterations):
        n_centers, n_reqs = sa_neighbor(centers, reqs)
        f_new = sa_fitness(n_centers, n_reqs)
        delta = f_new - f_cur
        if delta > 0 or random.random() < math.exp(delta / max(T, 1e-12)):
            centers, reqs = n_centers, n_reqs
            f_cur = f_new
            if f_new > f_best:
                best_c, best_r = n_centers, n_reqs
                f_best = f_new
        T *= cooling
    return best_c, best_r, -f_best

# Single SA run
init = sa_random_candidate()
sa_centers, sa_reqs, sa_best = simulated_annealing(init)
print("SA best fitness:", sa_best)

# # 10 runs summary
# SA_runs = []
# for run in range(1, 11):
#     random.seed(100 + run); np.random.seed(200 + run)
#     init = sa_random_candidate()
#     c_best, r_best, f_best = simulated_annealing(init)
#     SA_runs.append({"Run": run, "BestFitness": f_best, "Centers": c_best, "ReqAssign": r_best})
# SA_df = pd.DataFrame([{"Run": r["Run"], "BestFitness": r["BestFitness"]} for r in SA_runs])
# SA_df.to_csv(OUT_DIR / "sa_summary.csv", index=False)
# SA_df

SA best fitness: 259637.04878289835


In [29]:
import folium

def viz_assignments(assignments, title="Assignments", filename="map.html", use_road_network=False):
    """
    assignments: list of tuples (center_coords, courier_index, request_index)
    e.g., [( (lat_c, lon_c), courier_id, request_id ), ...]
    """
    # Create a base map
    m = folium.Map(location=[24.7136, 46.6753], zoom_start=10)  # Riyadh center (adjust as needed)

    # Add center and request markers
    for (center, courier_id, request_id) in assignments:
        folium.Marker(
            location=center,
            popup=f"Courier {courier_id} → Request {request_id}",
            icon=folium.Icon(color="blue", icon="truck", prefix='fa')
        ).add_to(m)

    # Save map
    m.save(filename)
    print(f"✅ Map saved to {filename}")
    return m

# Visualize the single SA solution
sa_assignments = [(center_locations[j], sa_centers[j], r) for r, j in enumerate(sa_reqs)]
_ = viz_assignments(sa_assignments, "SA Assignments", "sa_map.html", use_road_network=True)

✅ Map saved to sa_map.html


## Part 7 — Adaptive Simulated Annealing (ASA)

In [22]:

def adaptive_sa(initial, iterations=5000, T_init=100.0, base_cooling=0.995,
                target_accept=0.23, window=200, adapt_strength=0.05):
    centers, reqs = copy.deepcopy(initial[0]), copy.deepcopy(initial[1])
    best_c, best_r = centers, reqs
    f_cur = sa_fitness(centers, reqs)
    f_best = f_cur
    T = T_init
    accepts = 0; trials = 0

    for it in range(1, iterations+1):
        n_centers, n_reqs = sa_neighbor(centers, reqs)
        f_new = sa_fitness(n_centers, n_reqs)
        delta = f_new - f_cur
        accept = (delta >= 0) or (T > 1e-12 and random.random() < math.exp(delta / T))
        trials += 1
        if accept:
            accepts += 1
            centers, reqs = n_centers, n_reqs
            f_cur = f_new
            if f_new > f_best:
                best_c, best_r = n_centers, n_reqs
                f_best = f_new
        T *= base_cooling
        if it % window == 0 and trials > 0:
            acc_rate = accepts / trials
            T *= math.exp(adapt_strength * (target_accept - acc_rate))
            accepts = 0; trials = 0
    return best_c, best_r, f_best

ASA_runs = []
for run in range(1, 11):
    random.seed(314000 + run); np.random.seed(271800 + run)
    init = sa_random_candidate()
    c_best, r_best, f_best = adaptive_sa(init, iterations=5000)
    ASA_runs.append({"Run": run, "BestFitness": f_best, "Centers": c_best, "ReqAssign": r_best})

ASA_df = pd.DataFrame([{"Run": r["Run"], "BestFitness": r["BestFitness"]} for r in ASA_runs])
ASA_df.to_csv(OUT_DIR / "asa_summary.csv", index=False)
print("fittness=",-f_best)

fittness= 259637.04878289835


## Part 8 — Genetic Algorithm (GA) & Adaptive GA (AGA)

In [25]:

def repair_requests(reqs):
    reqs = list(reqs)
    counts = [reqs.count(j) for j in range(num_centers)]
    # 1) move from >MAX to <MIN
    over = [j for j in range(num_centers) if counts[j] > MAX_PER_CENTER]
    under_min = [j for j in range(num_centers) if counts[j] < MIN_PER_CENTER]
    guard = 0
    while over and under_min and guard < 10000:
        guard += 1
        donor    = random.choice(over)
        receiver = random.choice(under_min)
        idxs = [i for i,x in enumerate(reqs) if x == donor]
        if not idxs:
            over = [jj for jj in over if jj != donor]; continue
        idx = random.choice(idxs)
        reqs[idx] = receiver
        counts[donor] -= 1; counts[receiver] += 1
        if counts[donor] <= MAX_PER_CENTER: over = [jj for jj in over if jj != donor]
        if counts[receiver] >= MIN_PER_CENTER: under_min = [jj for jj in under_min if jj != receiver]
    # 2) >MAX to <MAX
    over = [j for j in range(num_centers) if counts[j] > MAX_PER_CENTER]
    under_max = [j for j in range(num_centers) if counts[j] < MAX_PER_CENTER]
    while over and under_max and guard < 20000:
        guard += 1
        donor, receiver = random.choice(over), random.choice(under_max)
        idxs = [i for i,x in enumerate(reqs) if x == donor]
        if not idxs:
            over = [jj for jj in over if jj != donor]; continue
        idx = random.choice(idxs)
        reqs[idx] = receiver
        counts[donor] -= 1; counts[receiver] += 1
        if counts[donor] <= MAX_PER_CENTER: over = [jj for jj in over if jj != donor]
        if counts[receiver] >= MAX_PER_CENTER: under_max = [jj for jj in under_max if jj != receiver]
    # 3) lift any <MIN by stealing from >MIN
    under_min = [j for j in range(num_centers) if counts[j] < MIN_PER_CENTER]
    donors = [j for j in range(num_centers) if counts[j] > MIN_PER_CENTER]
    while under_min and donors and guard < 30000:
        guard += 1
        receiver = under_min[0]
        needed = MIN_PER_CENTER - counts[receiver]
        moved = 0
        while moved < needed and donors:
            donor = random.choice(donors)
            idxs = [i for i,x in enumerate(reqs) if x == donor]
            if not idxs:
                donors = [jj for jj in donors if jj != donor]; continue
            idx = random.choice(idxs)
            reqs[idx] = receiver
            counts[donor] -= 1; counts[receiver] += 1
            moved += 1
            if counts[donor] <= MIN_PER_CENTER: donors = [jj for jj in donors if jj != donor]
        under_min = [j for j in range(num_centers) if counts[j] < MIN_PER_CENTER]
    return reqs

def ga_fitness(candidate):
    centers_asg, reqs_asg = candidate
    total = 0.0
    for r, j in enumerate(reqs_asg):
        d = centers_asg[j]
        total += P_cr[d, r] - LAMBDA_WEIGHT * (D_cc_prime[d, j] + D_c_prime_r[j, r])
    return total

def ga_random_candidate():
    centers = random.sample(range(num_couriers), num_centers)
    reqs = [random.randrange(num_centers) for _ in range(num_requests)]
    reqs = repair_requests(reqs)
    return centers, reqs

def ga_crossover(p1, p2):
    c1, r1 = p1; c2, r2 = p2
    idx = random.randrange(num_centers)
    new_c = c1.copy(); new_c[idx] = c2[idx]
    # fix duplicates
    missing = [d for d in range(num_couriers) if d not in new_c]
    seen = set()
    for i in range(num_centers):
        if new_c[i] in seen:
            new_c[i] = missing.pop()
        else:
            seen.add(new_c[i])
    pt = random.randrange(1, num_requests) if num_requests > 1 else 1
    new_r = r1[:pt] + r2[pt:]
    new_r = repair_requests(new_r)
    return new_c, new_r

def ga_mutate(candidate, mp=0.05):
    centers, reqs = copy.deepcopy(candidate)
    if num_centers >= 2 and random.random() < mp:
        i, j = random.sample(range(num_centers), 2)
        centers[i], centers[j] = centers[j], centers[i]
    for i in range(num_requests):
        if random.random() < mp:
            reqs[i] = random.randrange(num_centers)
    reqs = repair_requests(reqs)
    return centers, reqs

def tournament_select(pop, fits, k=3):
    pool = random.sample(list(zip(pop, fits)), k)
    return max(pool, key=lambda x: x[1])[0]

def run_ga(seed, pop_size=50, generations=200, mut_prob=0.05, elite=1):
    random.seed(seed); np.random.seed(seed)
    population = [ga_random_candidate() for _ in range(pop_size)]
    for _ in range(generations):
        fits = [ga_fitness(ind) for ind in population]
        new_pop = []
        # elitism
        elite_idx = int(np.argmax(fits))
        for _ in range(elite):
            new_pop.append(population[elite_idx])
        # rest
        while len(new_pop) < pop_size:
            p1 = tournament_select(population, fits)
            p2 = tournament_select(population, fits)
            child = ga_crossover(p1, p2)
            child = ga_mutate(child, mut_prob)
            new_pop.append(child)
        population = new_pop
    # best
    fits = [ga_fitness(ind) for ind in population]
    best_idx = int(np.argmax(fits))
    best = population[best_idx]
    return {"seed": seed, "fitness": float(fits[best_idx]), "best": best}

# GA 10 runs
GA_runs = [run_ga(seed) for seed in range(10)]
GA_df = pd.DataFrame([{"Run": i, "Seed": r["seed"], "BestFitness": -r["fitness"]} for i, r in enumerate(GA_runs, 1)])
GA_df.to_csv(OUT_DIR / "ga_summary.csv", index=False)
GA_df

,Run,Seed,BestFitness
0,1,0,257733.974825
1,2,1,257690.644734
2,3,2,256448.127350
3,4,3,256448.127350
4,5,4,256656.739751
5,6,5,257690.644734
6,7,6,256448.127350
7,8,7,256448.127350
8,9,8,256448.127350
9,10,9,254636.833574


**# Adaptive GA (AGA)**

In [35]:
def population_diversity(pop):
    req_sig = {"".join(map(str, ind[1])) for ind in pop}
    ctr_sig = {tuple(ind[0]) for ind in pop}
    return len(req_sig) + len(ctr_sig)

def run_aga(seed, pop_size=50, generations=200, mut_p_init=0.05, elite=1,
            stagnation_patience=25, div_target=0.30, mut_bounds=(0.01, 0.25)):
    random.seed(seed); np.random.seed(seed)
    population = [ga_random_candidate() for _ in range(pop_size)]
    fits = [ga_fitness(ind) for ind in population]
    best_idx = int(np.argmax(fits)); best_ind = population[best_idx]; best_fit = fits[best_idx]
    no_improve = 0; mut_p = mut_p_init; history = [best_fit]

    for gen in range(1, generations+1):
        div = population_diversity(population) / (2 * pop_size)
        if div < div_target:
            mut_p = min(mut_bounds[1], mut_p * 1.15)
        else:
            mut_p = max(mut_bounds[0], mut_p * 0.98)
        if no_improve >= stagnation_patience:
            mut_p = min(mut_bounds[1], max(mut_p, mut_p_init) * 1.5)
            no_improve = 0

        new_pop = []
        if elite > 0:
            elite_idx = int(np.argmax(fits))
            new_pop.append(population[elite_idx])

        while len(new_pop) < pop_size:
            p1 = tournament_select(population, fits)
            p2 = tournament_select(population, fits)
            child = ga_crossover(p1, p2)
            child = ga_mutate(child, mut_p)
            new_pop.append(child)

        population = new_pop
        fits = [ga_fitness(ind) for ind in population]
        cur_best_idx = int(np.argmax(fits))
        cur_best_fit = fits[cur_best_idx]
        if cur_best_fit > best_fit + 1e-9:
            best_fit = cur_best_fit
            best_ind = population[cur_best_idx]
            no_improve = 0
        else:
            no_improve += 1
        history.append(best_fit)

    return {"seed": seed, "fitness": float(best_fit), "best": best_ind, "history": history, "mut_p_final": mut_p}

# AGA 10 runs
AGA_runs = [run_aga(seed) for seed in range(10)]
AGA_df = pd.DataFrame([{"Run": i, "Seed": r["seed"], "BestFitness": -r["fitness"]} for i, r in enumerate(AGA_runs, 1)])
AGA_df.to_csv(OUT_DIR / "aga_summary.csv", index=False)
AGA_df

,Run,Seed,BestFitness
0,1,0,256656.739751
1,2,1,256112.172588
2,3,2,256448.127350
3,4,3,256448.127350
4,5,4,254429.744422
5,6,5,254429.744422
6,7,6,254429.744422
7,8,7,256448.127350
8,9,8,254429.744422
9,10,9,256448.127350
